<a href="https://colab.research.google.com/github/CarolineAndradeR/Python/blob/master/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

# Criar a sessão
spark = SparkSession.builder.appName("ConsultasSparkSQL").getOrCreate()

# Criar um DataFrame de exemplo
dados = [
    (1, "Alice", "TI", 3000),
    (2, "Bob", "RH", 2500),
    (3, "Carol", "TI", 4000),
    (4, "Daniel", "Financeiro", 3500),
    (5, "Ester", "RH", 2000)
]

colunas = ["id", "nome", "departamento", "salario"]

df = spark.createDataFrame(dados, colunas)
df.show()


+---+------+------------+-------+
| id|  nome|departamento|salario|
+---+------+------------+-------+
|  1| Alice|          TI|   3000|
|  2|   Bob|          RH|   2500|
|  3| Carol|          TI|   4000|
|  4|Daniel|  Financeiro|   3500|
|  5| Ester|          RH|   2000|
+---+------+------------+-------+



# **Filtros e Seleções**

In [5]:
# Seleção de colunas especificas
df.select("nome", "salario").show()

+------+-------+
|  nome|salario|
+------+-------+
| Alice|   3000|
|   Bob|   2500|
| Carol|   4000|
|Daniel|   3500|
| Ester|   2000|
+------+-------+



In [3]:
# Filtro (where)
df.filter(df.salario > 2500).show()

+---+------+------------+-------+
| id|  nome|departamento|salario|
+---+------+------------+-------+
|  1| Alice|          TI|   3000|
|  3| Carol|          TI|   4000|
|  4|Daniel|  Financeiro|   3500|
+---+------+------------+-------+



In [4]:
#Agrupamento por coluna
df.groupBy("departamento").avg("salario").show()

+------------+------------+
|departamento|avg(salario)|
+------------+------------+
|          TI|      3500.0|
|          RH|      2250.0|
|  Financeiro|      3500.0|
+------------+------------+



# **Spark SQL**

In [6]:
# Antes de rodar o SQL é necessário criar um Dataframe como tabela temporária
df.createOrReplaceTempView("funcionarios")

In [7]:
spark.sql("SELECT nome, salario FROM funcionarios").show()

+------+-------+
|  nome|salario|
+------+-------+
| Alice|   3000|
|   Bob|   2500|
| Carol|   4000|
|Daniel|   3500|
| Ester|   2000|
+------+-------+



In [8]:
spark.sql("SELECT * FROM funcionarios WHERE salario > 2500").show()

+---+------+------------+-------+
| id|  nome|departamento|salario|
+---+------+------------+-------+
|  1| Alice|          TI|   3000|
|  3| Carol|          TI|   4000|
|  4|Daniel|  Financeiro|   3500|
+---+------+------------+-------+



In [9]:
spark.sql("""
    SELECT departamento, AVG(salario) as media_salario
    FROM funcionarios
    GROUP BY departamento
""").show()

+------------+-------------+
|departamento|media_salario|
+------------+-------------+
|          TI|       3500.0|
|          RH|       2250.0|
|  Financeiro|       3500.0|
+------------+-------------+



# **Exercícios**

In [10]:
#Quais funcionários ganham menos de 3000?
df.filter(df.salario < 3000).show()

+---+-----+------------+-------+
| id| nome|departamento|salario|
+---+-----+------------+-------+
|  2|  Bob|          RH|   2500|
|  5|Ester|          RH|   2000|
+---+-----+------------+-------+



In [11]:
#Quais funcionários ganham menos de 3000?
spark.sql("SELECT * FROM funcionarios WHERE salario < 3000").show()

+---+-----+------------+-------+
| id| nome|departamento|salario|
+---+-----+------------+-------+
|  2|  Bob|          RH|   2500|
|  5|Ester|          RH|   2000|
+---+-----+------------+-------+



In [37]:
#Qual é o maior salário do departamento de RH?
from pyspark.sql.functions import max

df.filter(df.departamento=="RH").agg(max(df.salario)).show()

+------------+
|max(salario)|
+------------+
|        2500|
+------------+



In [35]:
#Qual é o maior salário do departamento de RH?
spark.sql("""
    SELECT max(salario) as maior_salario
    FROM funcionarios
    WHERE departamento = 'RH'
""").show()

+-------------+
|maior_salario|
+-------------+
|         2500|
+-------------+



In [38]:
#Liste todos os nomes em ordem decrescente de salário.
df.orderBy(df.salario.desc()).select("nome").show()

+------+
|  nome|
+------+
| Carol|
|Daniel|
| Alice|
|   Bob|
| Ester|
+------+



In [39]:
#Liste todos os nomes em ordem decrescente de salário.
spark.sql("""
    SELECT nome
    FROM funcionarios
    ORDER BY salario DESC
""").show()

+------+
|  nome|
+------+
| Carol|
|Daniel|
| Alice|
|   Bob|
| Ester|
+------+



In [53]:
#Qual o salário médio geral da empresa?
from pyspark.sql import functions as F

df.agg(F.avg("salario").alias("salario_medio")).show()

+-------------+
|salario_medio|
+-------------+
|       3000.0|
+-------------+



In [59]:
#Qual o salário médio geral da empresa?
spark.sql("""
    SELECT AVG(salario) as salario_medio
    FROM funcionarios
    """).show()

+-------------+
|salario_medio|
+-------------+
|       3000.0|
+-------------+



In [60]:
# Salário médio por departamento
df.groupBy("departamento").agg(F.avg("salario").alias("salario_medio")).show()

+------------+-------------+
|departamento|salario_medio|
+------------+-------------+
|          TI|       3500.0|
|          RH|       2250.0|
|  Financeiro|       3500.0|
+------------+-------------+



In [67]:
# Salário médio por departamento
spark.sql("""
    SELECT departamento, AVG(salario) as salario_medio
    FROM funcionarios
    GROUP BY departamento
""").show()

+------------+-------------+
|departamento|salario_medio|
+------------+-------------+
|          TI|       3500.0|
|          RH|       2250.0|
|  Financeiro|       3500.0|
+------------+-------------+



In [70]:
# Maior e menor salário da empresa
from pyspark.sql.functions import min
df.agg(min("salario"),max("salario")).show()

+------------+------------+
|min(salario)|max(salario)|
+------------+------------+
|        2000|        4000|
+------------+------------+



In [71]:
#Maior e menor salário da empresa
spark.sql("""
    select min(salario) as menor, max(salario) as maior
    from funcionarios
""").show()

+-----+-----+
|menor|maior|
+-----+-----+
| 2000| 4000|
+-----+-----+



In [76]:
# Quantos funcionários existem em cada cargo?
from pyspark.sql.functions import count
df.groupBy("departamento").agg(count("nome").alias("quantidade_funcionarios")).show()

+------------+-----------------------+
|departamento|quantidade_funcionarios|
+------------+-----------------------+
|          TI|                      2|
|          RH|                      2|
|  Financeiro|                      1|
+------------+-----------------------+



In [75]:
# Quantos funcionários existem em cada cargo?
spark.sql("""
    select departamento, count(nome) as quantidade
    from funcionarios
    group by 1
""").show()

+------------+----------+
|departamento|quantidade|
+------------+----------+
|          TI|         2|
|          RH|         2|
|  Financeiro|         1|
+------------+----------+



In [82]:
#Soma dos salários de todos os funcionários de TI
df.filter(df.departamento=="TI").agg(F.sum(df.salario).alias("salarios_ti")).show()

+-----------+
|salarios_ti|
+-----------+
|       7000|
+-----------+



In [83]:
#Soma dos salários de todos os funcionários de TI
spark.sql("""
    select sum(salario) as salarios_ti
    from funcionarios
    where departamento = "TI"
""").show()

+-----------+
|salarios_ti|
+-----------+
|       7000|
+-----------+



In [88]:
# Média salarial considerando apenas salários acima de 5.000
df.filter(df.salario>5000).agg(F.avg(df.salario).alias("media_salarial")).show()

+--------------+
|media_salarial|
+--------------+
|          NULL|
+--------------+

